# <p style="text-align: center;">MIS 382N: Advanced Predictive Modeling</p>
# <p style="text-align: center;">Assignment 5</p>
## <p style="text-align: center;">Total points: 50 </p>
## <p style="text-align: center;">Due: Mon, November 28</p>



Your homework should be written in a **Jupyter notebook**. Please submit **only one** ipynb file from each group, and include the names of all the group members. Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting.

# Problem 1 - Random Forest vs Boosting - Regression (15pts)

In this question, we will compare performance of different ensemble methods for regression problems: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html), [Gradient Boosting Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (GBR), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor). Board game data set from DataQuest will be used (you can download data from Canvas: 'games.csv').

1. (1) Load the data, (2) remove duplicate rows, (3) remove features of type string (object in pandas), and (4) replace missing values by mean of each column. Then, partition data into features (X) and the target label (y) for regression task. We want to predict the *average_rating*. Use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split data into training and testing: test_size=0.33, random_state=42. (1pt)

2. Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) to predict average_rating. Find the best parameters (including *n_estimators*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report the accuracy of your model in terms of RMSE. (4pts)

3. Use [Gradient Boosting Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (GBR), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor) for predicting the targets. Again, find the best parameters (including *n_estimators,* and* learning_rate*), and report corresponding RMSE for each algorithm. (8pts)

4. Which model did you expect to be more accurate in predicting the targets? Why? Did your observation match this expectation? (2pts)

In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import (RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor)

# Part 1

In [34]:
Games = pd.read_csv('C:\Users\Owner\Documents\MSBA\Datasets\games.csv')
#drop duplicates
Games = Games.drop_duplicates()
#exclude columns of type object i.e. strings
Games = Games.select_dtypes(exclude=['object'])
#replace missing values with mean of each column
Games = Games.fillna(Games.mean())
# split df into target and features
Target = Games['average_rating']
Features = Games.drop(['average_rating'], axis=1)
# split into training and testing set
x_train, x_test, y_train, y_test = train_test_split(Features, Target, test_size=0.33, random_state=42)

# Part 2

In [40]:
RFR = RandomForestRegressor()
# create paramter grid for exhaustive search
param_grid = { 'n_estimators' : [2,6,10,14,18], 'max_features' : ['auto', 'sqrt', 'log2'] }
#make model
cv_rfv = GridSearchCV(estimator = RFR, param_grid=param_grid, cv=5)
#fit model that does exhaustive search for best params
cv_rfv.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [2, 6, 10, 14, 18], 'max_features': ['auto', 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [42]:
print "{}:{}".format("Best parameters", cv_rfv.best_params_)
print "{}{}".format("Best score:", cv_rfv.best_score_)

Best parameters:{'max_features': 'sqrt', 'n_estimators': 18}
Best score:0.942559933353


In [44]:
#make model with best parameters
RFR = RandomForestRegressor(n_estimators = 18, max_features = 'sqrt')
model = RFR.fit(x_train, y_train)
pred_y = model.predict(x_test)
test_MSE = mean_squared_error(y_test, pred_y)
print 'RMSE:', np.sqrt(test_MSE)


RMSE: 1.01349835497


# Part 3

First we'll do gradient boosting regressor and then Adaboost

In [48]:
GBR = GradientBoostingRegressor()
# create paramter grid for exhaustive search
param_grid_GBR = { 'learning_rate' : [.001,.01,.1,1], 'loss' : ['ls', 'lad', 'huber', 'quantile'], 'n_estimators' : [100,150,200], 
             'max_features' : ['auto', 'sqrt', 'log2']}
#make model
cv_GBR = GridSearchCV(estimator = GBR, param_grid=param_grid_GBR, cv=5)
#fit model that does exhaustive search for best params
cv_GBR.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 150, 200], 'loss': ['ls', 'lad', 'huber', 'quantile'], 'learning_rate': [0.001, 0.01, 0.1, 1], 'max_features': ['auto', 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [49]:
print "{}:{}".format("Best parameters", cv_GBR.best_params_)
print "{}{}".format("Best score:", cv_GBR.best_score_)

Best parameters:{'max_features': 'auto', 'loss': 'ls', 'learning_rate': 0.1, 'n_estimators': 200}
Best score:0.890112249897


In [50]:
#make model with best parameters
GBR = GradientBoostingRegressor(max_features='auto', loss='ls', learning_rate=0.1, n_estimators=200)
model = GBR.fit(x_train, y_train)
pred_y = model.predict(x_test)
test_MSE = mean_squared_error(y_test, pred_y)
print 'RMSE:', np.sqrt(test_MSE)

RMSE: 1.01565867416


## Adaboost

In [51]:
ABR = AdaBoostRegressor()
# create paramter grid for exhaustive search
param_grid_ABR = { 'learning_rate' : [.0001,.001,.01,.1,1], 'loss' : ['linear', 'exponential', 'square'], 'n_estimators' : [25,50,100]}
#make model
cv_ABR = GridSearchCV(estimator = ABR, param_grid=param_grid_ABR, cv=5)
#fit model that does exhaustive search for best params
cv_ABR.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [25, 50, 100], 'loss': ['linear', 'exponential', 'square'], 'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [52]:
print "{}:{}".format("Best parameters", cv_ABR.best_params_)
print "{}{}".format("Best score:", cv_ABR.best_score_)

Best parameters:{'n_estimators': 100, 'loss': 'exponential', 'learning_rate': 0.1}
Best score:0.86291001213


In [53]:
#make model with best parameters
ABR = AdaBoostRegressor(n_estimators = 100, loss = 'exponential', learning_rate=0.1)
model = ABR.fit(x_train, y_train)
pred_y = model.predict(x_test)
test_MSE = mean_squared_error(y_test, pred_y)
print 'RMSE:', np.sqrt(test_MSE)

RMSE: 1.13231974067


# Part 4

I thought going into it that Gradient Boosting Regressor (GBR) would perform the best simply because it's a very powerful model.

# Problem 2 - Random Forest vs Boosting - Classification (15 pts)
In this question, we will compare performance of different ensemble methods for classification problems: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier). [Spam Classification Data](https://archive.ics.uci.edu/ml/datasets/Spambase) of UCI will be used (you can download data from Canvas: 'spam_uci.csv'). Don't worry about column names. The last column represents target label, 1 if spam and zero otherwise.

1. Load the data and partition it into features (X) and the target label (y) for classification task. Then, use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split data into training and testing: test_size=0.33, random_state=42. (1pt)

2. Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to classify whether an email is spam. Find the best parameters (including *n_estimators* and *criterion*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report your testing accuracy ([accuracy_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)) and [roc_auc_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score). You will need [predict_proba](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.predict_proba) for roc_auc_score. (4pts)

3. Use [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) for the spam classification problem. Again, find the best parameters (including *n_estimators, learning_rate,* and *max_depth (GBDT only)*), and report corresponding accuracy_score and roc_auc_score on the test data for each algorithm. (8pts)

4. Point out one advantage and one disadvantage of Random Forest compared to GBDT (2pts)

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import (accuracy_score,roc_auc_score)
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier)

# Part 1

In [5]:
Spam_uci = pd.read_csv('C:\Users\Owner\Documents\MSBA\Datasets\spam_uci.csv')
# split df into target and features
Target = Spam_uci[[-1]]
Features = Spam_uci.drop(['57'], axis=1)
# split into training and testing set
x_train, x_test, y_train, y_test = train_test_split(Features, Target, test_size=0.33, random_state=42)

In [11]:
x_train

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,47,48,49,50,51,52,53,54,55,56
3560,3560,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.237,0.000,0.000,0.000,0.000,1.800,9,36
2589,2589,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.120,0.000,0.241,0.000,0.000,3.541,26,85
3062,3062,0.17,0.00,0.35,0.0,0.53,0.00,0.00,0.35,0.00,...,0.00,0.071,0.143,0.000,0.000,0.000,0.000,2.464,80,244
4435,4435,0.00,0.00,0.59,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.039,0.235,0.000,0.471,0.000,0.039,3.659,70,333
3083,3083,0.00,14.28,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.000,0.000,0.000,0.000,0.000,1.800,5,9
3048,3048,0.00,0.00,0.00,0.0,1.81,0.00,0.00,0.00,0.00,...,0.00,0.000,0.615,0.000,0.000,0.000,0.000,1.388,5,25
3089,3089,1.00,0.00,1.00,0.0,1.00,1.00,0.00,0.00,0.00,...,0.00,0.000,0.324,0.000,0.487,0.000,0.000,2.291,22,55
2807,2807,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.000,0.000,0.000,0.000,0.000,2.275,8,91
2706,2706,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,1.38,0.000,0.000,0.000,0.000,0.000,0.000,1.666,9,35
1278,1278,0.00,0.89,1.15,0.0,0.12,0.00,0.00,0.12,0.25,...,0.00,0.000,0.185,0.000,0.370,0.061,0.000,2.878,84,475


# Part 2

In [4]:
RFC = RandomForestClassifier()
# create paramter grid for exhaustive search
param_grid = { 'n_estimators' : [5,7,10,13,15], 'criterion' : ['gini', 'entropy'] }
#make model
cv_rfc = GridSearchCV(estimator = RFC, param_grid=param_grid, cv=5)
#fit model that does exhaustive search for best params
cv_rfc.fit(x_train, y_train)

print "{}:{}".format("Best parameters", cv_rfc.best_params_)
print "{}{}".format("Best score:", cv_rfc.best_score_)

IndexError: too many indices for array

# Question 3 - Matrix Factorization for Rating Prediction (20pts)

The movielens dataset contains 1 million movie ratings from several thousand users. We will be using *k*-rank matrix factorization to estimate this dataset as the product $X=UV^T$, where *U* and *V* have only $k$ columns.

1) You can download the movielens 1M dataset from https://datahub.io/dataset/movielens, but for this problem use the data available on Canvas. It has been split into training and test sets, and converted to matrix format where the rows correspond to users and the columns to movies. Note that most of the entries are NaNs, indicating that these ratings are missing. An extra file, lens1m_361M_titles.csv, has been added so you can check out specific movies if you're curious.

2) Scikit-learn is a little behind for recommender systems, and doesn't have any method to factorize matrices with missing data. Which means you get to code it! Slide 22 of the 'apa large scale learning' lecture notes has the equations for stochastic gradient descent on *U* and *V*. You will have to:
* Set up initial guesses for the *U* and *V* matrices. I suggest small random values.
* Find a suitable learning rate for the descent. A learning rate that is too large will probably blow up, like in HW3 problem 1.
* Come up with a stopping policy
* Code the descent algorithm (5 pts)

3) Using your SGD algorithm, apply 2-rank matrix factorization on the filled training matrix. Calculate the RMSE of this model on the training data and on the test data (separately). The optimal score on the training data is around .86 RMSE; your version of gradient descent must go at least below .91 RMSE. (5 pts)

4) You should notice some overfitting. Because matrix factorization learns separate scores for each movie, a movie with very few reviews may be easily overfit. You may want to only predict ratings when you have enough information to reach a good conclusion. Recalculate the RMSE on the test data, specifically for movies with at least 50 reviews (don't retrain the models). Also report the percent of movies that are still included (after cutting those with < 50 reviews), and the percent of test ratings that are still included. (5 pts)

5) Repeat steps 3 and 4 with 5-rank factorization. Display training and test RMSE. (5 pts)

Hints:  
The numpy function *nanmean* is helpful for RMSE calculation.  
The descent algorithm will probably run for at least several minutes.

In [8]:
import numpy as np
import pandas as pd
titles = pd.read_csv('lens1m_361M_titles.csv')
test_X = np.load('lens1m_361M_test.npy')
train_X = np.load('lens1m_361M_train.npy')